## Performance and risk metric computations for active funds

by DC LIM

In this Jupyter notebook, we run through a basic analysis of performance and risk metric computations for active funds, e.g. Sharpe ratio, tracking error, VaR, max drawdown with monte carlo simulations and so on. We will also conduct factor analysis to deduce the fund manager's stock selection method. 

For this exercise, we use the "Franklin DynaTech Fund Class A" fund for analysis. One of the fund's benchmarks is the S&P 500 index, which we will also use to evaluate the fund's performance. Note that the computations can be done for any fund with returns data captured on Yahoo Finance. 

In [1]:
#import the relevant libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
from scipy.optimize import minimize
import empyrical as emp
import statsmodels.api as sm
import yfinance as yf
import ipywidgets as widgets
from IPython.display import display

In [6]:
# Specify some parameters such as number of years of data to get and latest closing price that 
# will be used in subsequent calculations

years = 20
latest_closing_price = 120

Get the returns data for both the fund and its benchmark index

In [7]:
tickerSymbol = 'FKDNX'

#get data on this ticker
tickerData = yf.Ticker(tickerSymbol)

#get the historical prices for this ticker over the last 20 years
df = tickerData.history(period='1d', start='2000-8-30', end = '2020-8-30')

In [8]:
tickerSymbol2 = '^GSPC'

#get data on this ticker
tickerData2 = yf.Ticker(tickerSymbol2)

#get the historical prices for this ticker over the last 20 years
df2 = tickerData2.history(period='1d', start='2000-8-30', end = '2020-8-30')

In [9]:
df.head()

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2000-08-29,21.32,21.32,21.32,21.32,0,0.0,0
2000-08-30,21.34,21.34,21.34,21.34,0,0.0,0
2000-08-31,21.57,21.57,21.57,21.57,0,0.0,0
2000-09-01,21.66,21.66,21.66,21.66,0,0.0,0
2000-09-05,21.35,21.35,21.35,21.35,0,0.0,0


In [10]:
# Get the daily returns from the changes in the daily closing prices
rets = df["Close"]
rets = rets.pct_change()
rets = rets.iloc[1:] 
rets

Date
2000-08-30    0.000938
2000-08-31    0.010778
2000-09-01    0.004172
2000-09-05   -0.014312
2000-09-06   -0.019672
                ...   
2020-08-24   -0.000417
2020-08-25    0.011353
2020-08-26    0.025093
2020-08-27   -0.005637
2020-08-28    0.005264
Name: Close, Length: 5031, dtype: float64

In [11]:
# Compute the returns for the benchmark index using closing prices
rets2 = df2["Close"]
rets2 = rets2.pct_change()
rets2 = rets2.iloc[1:] 
rets2

Date
2000-08-30   -0.004802
2000-08-31    0.010043
2000-09-01    0.002036
2000-09-05   -0.009002
2000-09-06   -0.009840
                ...   
2020-08-24    0.010044
2020-08-25    0.003596
2020-08-26    0.010196
2020-08-27    0.001673
2020-08-28    0.006733
Name: Close, Length: 5031, dtype: float64

### Fund performance metrics

In [12]:
# Calculate the annualised return over the entire period

def annualise_rets(r, periods_per_year):
    """
    Annualises a set of returns
    """
    compounded_growth = (1+r).prod()
    n_periods = r.shape[0]
    return compounded_growth**(periods_per_year/n_periods)-1

annual_return = annualise_rets(rets, 252)
annual_return

0.09225656555683792

In [13]:
# Calculate the annualised return of the benchmark

benchmark_return = annualise_rets(rets2, 252)
benchmark_return

0.04313194006545995

In [14]:
# Calculate the annualised Sharpe Ratio of the fund, assuming the risk free rate is 1%

def sharpe_ratio(r, riskfree_rate, periods_per_year):
    """
    Computes the annualized sharpe ratio of a set of returns
    """
    # convert the annual riskfree rate to per period
    rf_per_period = (1+riskfree_rate)**(1/periods_per_year)-1
    excess_ret = r - rf_per_period
    ann_ex_ret = annualise_rets(excess_ret, periods_per_year)
    ann_vol = annualise_vol(r, periods_per_year)
    return ann_ex_ret/ann_vol

def annualise_vol(r, periods_per_year):
    """
    Annualises the vol of a set of returns
    """
    return r.std()*(periods_per_year**0.5)

fund_sharpe_ratio = sharpe_ratio(rets, 0.01, 252)
fund_sharpe_ratio

0.39728300255003

In [15]:
# Calculate the annualised Sharpe Ratio of the benchmark, assuming the risk free rate is 1%
benchmark_sharpe_ratio = sharpe_ratio(rets2, 0.01, 252)
benchmark_sharpe_ratio

0.16546639417919418

In [16]:
# Calculate the Treynor ratio of the fund, assuming a risk-free rate of 1%

def beta(returns, market):
    # Create a matrix of [returns, market]
    m = np.matrix([returns, market])
    # Return the covariance of m divided by the standard deviation of the market returns
    return np.cov(m)[0][1] / np.std(market)

fund_beta = beta(np.array(rets), np.array(rets2))

fund_treynor_ratio = (annual_return - 0.01)/fund_beta
fund_treynor_ratio

6.984070178219755

In [17]:
# Calculate the Treynor ratio of the benchmark, assuming a risk-free rate of 1%

benchmark_treynor_ratio = (benchmark_return - 0.01)/1
benchmark_treynor_ratio

0.03313194006545995

In [18]:
# Calculate the Sortino ratio of the fund, assuming a risk-free rate of 1%

def sortino_ratio(returns, risk_free_rate = 0, target=0):
    """
    Takes a set of returns with type series, calculate the downside deviation of the expected returns 
    by taking the difference between each period’s return and the target return. If a period’s return is 
    greater than the target return, the difference is set to 0. Then, we square the value of the 
    difference. Next, we calculate the average of all squared differences. The square root of the average 
    is the downside deviation.
    """
    df = pd.DataFrame(returns)
    df['downside_returns'] = 0
    df.loc[df[df.columns[0]] < target, 'downside_returns'] = df[df.columns[0]]**2
    expected_return = df[df.columns[0]].mean()
    down_stdev = np.sqrt(df['downside_returns'].mean())
    sortino_ratio = (expected_return - risk_free_rate)/down_stdev
    return sortino_ratio
    
fund_sortino_ratio = sortino_ratio(rets, 0.01, 0)
fund_sortino_ratio

-1.046854663386573

In [19]:
# Calculate the Sortino ratio of the benchmark, assuming a risk-free rate of 1%
benchmark_sortino_ratio = sortino_ratio(rets2, 0.01, 0)
benchmark_sortino_ratio

-1.093974888771736

The fund returns a good 10.3% over 20 years, compared to 4.8% for the benchmark index. In terms of various risk-adjusted return metrics, it also fares better than the benchmark, in particular for the Treynor ratio.

### Fund risk metrics

In [20]:
# Calculate the annualised volatility over the entire period
annual_vol = annualise_vol(rets, 252)
annual_vol

0.20500395377284455

In [21]:
# Calculate the max drawdown
Max_drawdown = emp.max_drawdown(rets)
Max_drawdown

-0.5044247787610621

In [22]:
# Calculate the Value at Risk of the fund (reflected as a positive number)

def var_historic(r, level=5):
    """
    Returns the historic Value at Risk at a specified level
    i.e. returns the number such that "level" percent of the returns
    fall below that number, and the (100-level) percent are above
    """
    if isinstance(r, pd.DataFrame):
        return r.aggregate(var_historic, level=level)
    elif isinstance(r, pd.Series):
        return -np.percentile(r, level)
    else:
        raise TypeError("Expected r to be a Series or DataFrame")


VaR = var_historic(rets, level = 5)
VaR

0.020085071629639795

In [23]:
# Calculate the Conditional Value at Risk of the fund (reflected as a positive number)

def cvar_historic(r, level=5):
    """
    Computes the Conditional VaR of Series or DataFrame
    """
    if isinstance(r, pd.Series):
        is_beyond = r <= -var_historic(r, level=level)
        return -r[is_beyond].mean()
    elif isinstance(r, pd.DataFrame):
        return r.aggregate(cvar_historic, level=level)
    else:
        raise TypeError("Expected r to be a Series or DataFrame")
        
CVaR = cvar_historic(rets, level = 5)
CVaR

0.029874830197363263

On any given day, there is a 5% chance that the fund would lose 2%, and in worse cases beyond that, the losses would average 3%.

In [24]:
# Calculate tracking error of the fund compared to the benchmark


tr = np.sqrt(((1/years)*(rets - rets2)**2).sum())
tr

0.0848515940517151

Tracking error of 8.49% is rather large, indicating substantial active management by the fund manager.

### Monte Carlo simulation of risk metrics

Here, we generate monte carlo simulations of the fund's prices over the next x years with n scenarios, coupled with the corresponding VaR computation for each scenario. The VaR estimates across the n scenarios are then compiled and reflected in a histogram.

In [25]:
def return_var(n_scenarios=1000, x_years = 10, mu=0.05, sigma=0.3, level = 5, y_max = 100, steps_per_year=252):
    """
    Draw the results of a stock price evolution under a Geometric Brownian Motion model, where mu is the annualised 
    expected return (drift), sigma the stock price volatility. VaR will be computed for each scenario. 
    """
    
    def gbm(x_years = 10, n_scenarios=1000, mu=0.05, sigma=0.3, steps_per_year=252, start = latest_closing_price):
        
        """
        Evolution of Geometric Brownian Motion trajectories, such as for Stock Prices through Monte Carlo
        :param x_years:  The number of years to generate data for
        :param n_paths: The number of scenarios/trajectories
        :param mu: Annualized Drift, e.g. Market Return
        :param sigma: Annualized Volatility
        :return: a numpy array of n_paths columns and n_years*steps_per_year rows
        """
        
        dt = 1/steps_per_year
        n_steps = int(x_years*steps_per_year) + 1
        # the standard way ...
        # rets_plus_1 = np.random.normal(loc=mu*dt+1, scale=sigma*np.sqrt(dt), size=(n_steps, n_scenarios))
        # without discretization error ...
        rets_plus_1 = np.random.normal(loc=(1+mu)**dt, scale=(sigma*np.sqrt(dt)), size=(n_steps, n_scenarios))
        rets_plus_1[0] = 1
        ret_val = start*pd.DataFrame(rets_plus_1).cumprod() 
        returns = pd.DataFrame(rets_plus_1-1)
        return ret_val, returns
    
    start = latest_closing_price
    model = gbm(x_years = x_years, n_scenarios=n_scenarios, mu=mu, sigma=sigma, steps_per_year = steps_per_year)
    prices = model[0]
    gbm_rets = model[1]
    var = var_historic(gbm_rets, level)
    
    # Plot the timeseries for prices and a histogram for VaR side by side
    fig, (prices_ax, var_ax) = plt.subplots(nrows=1, ncols=2, sharey=False, gridspec_kw={'width_ratios':[3,2]}, figsize=(30, 12))
    plt.subplots_adjust(wspace=0.0)
    
    # Plot the timeseries for prices
    prices.plot(ax=prices_ax, color="blue", legend = False, alpha = 0.5, linewidth=2, fontsize = 18)
    prices_ax.set_title('Price changes over time', fontsize = 24)
    prices_ax.axhline(y=100, ls=":", color="black")
    y_max = prices.values.max()*y_max/100
        
    # Plot a histogram for VaR as an inset axes over the main axes
    var.plot.hist(ax=var_ax, ec='w', color = "indianred", orientation='vertical', fontsize = 18)
    var_ax.set_title('VaR histogram', fontsize=24)
    
rv_controls = widgets.interactive(return_var,
                                   x_years = (0, 20, 1),
                                   n_scenarios=widgets.IntSlider(min=1, max=10000, step=100, value=500), 
                                   mu=(-0.2, +.2, .01),
                                   sigma=(0, .5, .05),
                                   level = (0, 20, 1),
                                   steps_per_year = (0, 252, 1),
                                   y_max=widgets.IntSlider(min=0, max=100, step=1, value=50,
                                                          description = "Zoom Y Axis")
)
display(rv_controls)

interactive(children=(IntSlider(value=500, description='n_scenarios', max=10000, min=1, step=100), IntSlider(v…

Here, we generate monte carlo simulations of the fund's prices over the next x years with n scenarios, coupled with the corresponding max drawdown computation for each scenario. The max drawdown estimates across the n scenarios are then compiled and reflected in a histogram.

In [26]:
def return_maxdd(n_scenarios=1000, x_years = 10, mu=0.05, sigma=0.15, y_max = 100, steps_per_year=252):
    """
    Draw the results of a stock price evolution under a Geometric Brownian Motion model, where mu is the annualised 
    expected return (drift), sigma the stock price volatility. Max drawdown will be computed for each scenario. 
    """
    
    def gbm(x_years = 10, n_scenarios=1000, mu=0.05, sigma=0.15, steps_per_year=252, start = latest_closing_price):
        
        """
        Evolution of Geometric Brownian Motion trajectories, such as for Stock Prices through Monte Carlo
        :param x_years:  The number of years to generate data for
        :param n_paths: The number of scenarios/trajectories
        :param mu: Annualized Drift, e.g. Market Return
        :param sigma: Annualized Volatility
        :return: a numpy array of n_paths columns and n_years*steps_per_year rows
        """
        
        dt = 1/steps_per_year
        n_steps = int(x_years*steps_per_year) + 1
        # the standard way ...
        # rets_plus_1 = np.random.normal(loc=mu*dt+1, scale=sigma*np.sqrt(dt), size=(n_steps, n_scenarios))
        # without discretization error ...
        rets_plus_1 = np.random.normal(loc=(1+mu)**dt, scale=(sigma*np.sqrt(dt)), size=(n_steps, n_scenarios))
        rets_plus_1[0] = 1
        ret_val = start*pd.DataFrame(rets_plus_1).cumprod() 
        returns = pd.DataFrame(rets_plus_1-1)
        return ret_val, returns
    
    start = latest_closing_price
    model = gbm(x_years = x_years, n_scenarios=n_scenarios, mu=mu, sigma=sigma, steps_per_year = steps_per_year)
    prices = model[0]
    gbm_rets = model[1]
    maxdd = emp.max_drawdown(gbm_rets)
    
    # Plot the timeseries for prices and a histogram for VaR side by side
    fig, (prices_ax, maxdd_ax) = plt.subplots(nrows=1, ncols=2, sharey=False, gridspec_kw={'width_ratios':[3,2]}, 
                                              figsize=(30, 12))
    plt.subplots_adjust(wspace=0.0)
    
    # Plot the timeseries for prices
    prices.plot(ax=prices_ax, color="blue", legend = False, alpha = 0.5, linewidth=2, fontsize = 18)
    prices_ax.set_title('Price changes over time', fontsize = 24)
    prices_ax.axhline(y=100, ls=":", color="black")
    y_max = prices.values.max()*y_max/100
        
    # Plot a histogram for VaR as an inset axes over the main axes
    maxdd.plot.hist(ax=maxdd_ax, ec='w', color = "indianred", orientation='vertical', fontsize = 18)
    maxdd_ax.set_title('Max drawdown histogram', fontsize=24)
    
ret_maxdd_controls = widgets.interactive(return_maxdd,
                                   x_years = (0, 20, 1),
                                   n_scenarios=widgets.IntSlider(min=1, max=10000, step=100, value=500), 
                                   mu=(-0.2, +.2, .01),
                                   sigma=(0, .5, .05),
                                   steps_per_year = (0, 252, 1),
                                   y_max=widgets.IntSlider(min=0, max=100, step=1, value=50,
                                                          description = "Zoom Y Axis")
)
display(ret_maxdd_controls)

interactive(children=(IntSlider(value=500, description='n_scenarios', max=10000, min=1, step=100), IntSlider(v…

### Applying factor analysis to the fund returns

We conduct factor analysis by regressing the returns of the fund against the returns of the Fama/French 5 factor model. Data for the latter is obtained from the website of Kenneth R. French.

The Fama/French 5 factors (2x3) are constructed using the 6 value-weight portfolios formed on size and book-to-market, the 6 value-weight portfolios formed on size and operating profitability, and the 6 value-weight portfolios formed on size and investment. (See the description of the 6 size/book-to-market, size/operating profitability, size/investment portfolios in the link below.)

http://mba.tuck.dartmouth.edu/pages/faculty/ken.french/Data_Library/f-f_5_factors_2x3.html 

In [27]:
# Read in the FF data

ff = pd.read_csv("data/F-F_Research_Data_5_Factors_2x3_daily.csv", parse_dates=True, header=0, index_col=0)
ff.index = pd.to_datetime(ff.index, format="%Y%m%d").to_period('D')
ff = ff['2000-8-30':'2020-8-30']/100

In [28]:
ff.head()

,Mkt-RF,SMB,HML,RMW,CMA,RF
2000-08-30,-0.0011,0.0056,0.0015,-0.0105,-0.0012,0.00022
2000-08-31,0.0119,0.0000,-0.0057,-0.0049,-0.0025,0.00022
2000-09-01,0.0038,0.0014,-0.0037,-0.0053,-0.0013,0.00025
2000-09-05,-0.0097,0.0031,0.0131,0.0037,0.0055,0.00025
2000-09-06,-0.0123,0.0048,0.0201,0.0111,0.0096,0.00025


In [29]:
# Calculate the excess returns of the fund over the risk free rate
fund_excess = np.array(rets - ff['RF'].values)

# Create a dataframe with the excess returns and the factors from the ff dataset
ff["Constant"] = 1
ff = ff.drop('RF', axis = 1)
ff.columns = ["Mkt-RF", "Size", "Value", "Profitability", "Investment", "RF"]
ff

,Mkt-RF,Size,Value,Profitability,Investment,RF
2000-08-30,-0.0011,0.0056,0.0015,-0.0105,-0.0012,1
2000-08-31,0.0119,0.0000,-0.0057,-0.0049,-0.0025,1
2000-09-01,0.0038,0.0014,-0.0037,-0.0053,-0.0013,1
2000-09-05,-0.0097,0.0031,0.0131,0.0037,0.0055,1
2000-09-06,-0.0123,0.0048,0.0201,0.0111,0.0096,1
...,...,...,...,...,...,...
2020-08-24,0.0092,0.0022,0.0258,0.0108,0.0061,1
2020-08-25,0.0036,-0.0001,-0.0063,-0.0045,-0.0069,1
2020-08-26,0.0101,-0.0166,-0.0174,-0.0009,-0.0084,1
2020-08-27,0.0018,-0.0003,0.0099,0.0055,-0.0046,1


In [30]:
fund_excess

array([ 0.00071809,  0.01055788,  0.00392246, ...,  0.02509286,
       -0.00563652,  0.00526358])

In [31]:
lm = sm.OLS(fund_excess, ff).fit()
lm.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.900
Model:                            OLS   Adj. R-squared:                  0.900
Method:                 Least Squares   F-statistic:                     9016.
Date:                Sun, 04 Oct 2020   Prob (F-statistic):               0.00
Time:                        15:29:21   Log-Likelihood:                 20528.
No. Observations:                5031   AIC:                        -4.104e+04
Df Residuals:                    5025   BIC:                        -4.100e+04
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
Mkt-RF            0.9361      0.005    180.080      0.000       0.926       0.946
Size              0.1192      0.010     11.765      0.000       0.099       0.139
Value            -0.3332      0.009    -35.763      0.000      -0.351      -0.315
Profitability    -0.1885      0.013    -14.197      0.000      -0.215      -0.162
Investment       -0.0974      0.017     -5.827      0.000      -0.130      -0.065
RF                0.0002   5.78e-05      2.839      0.005    5.08e-05       0.000
==============================================================================
Omnibus:                      725.531   Durbin-Watson:                   1.933
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            10830.899
Skew:                          -0.047   Prob(JB):                         0.00
Kurtosis:                      10.187   Cond. No.                         305.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

Based on the size of the variable coefficients, the fund manager has a tilt towards investing in smaller, growth companies that are less profitable and more aggressive in their investments. This matches the style of an investment fund that is targeted at tech stocks.
This seems to match the publicly available information on the fund's holdings. E.g. it among its top 10 holdings, there is SEA Ltd and ServiceNow, both of which are currently loss-making start-ups. Its top holding is Amazon, which until the recent years had been heavily loss-making since inception.